This is to create the spiking dataset for the EMG and DVS sensor fusion. The raw data can be found [here](https://zenodo.org/record/3228846#.XZSWWuczYWo) on zenodo. For practical reasons a stripped down version of the dataset with only emg and dvs can be accessed [here](https://www.dropbox.com/sh/fd5zeeu0yl04nxd/AADeBlpdjtnxdXF1fkvJj8v4a?dl=0) and that is what is used by this notebook to create the dataset, simply extract and provide the path in "data_dir". What the notebook does is extract and arrange the data taking care of the syncronization between EMG samples and DVS sample. Also the script is usefull if we wan to change the spikify parameters in the future.

In [15]:
import os
import sys
import wget
import numpy as np
import pickle as pkl

import matplotlib.pyplot as plt
%matplotlib inline

# local files
from converter import aedat2numpy
from utils import Person, signal_to_spike_refractory, find_trigger

if not os.path.exists('rawData'):
    os.makedirs('rawData')
    
if not os.path.exists('data'):
    os.makedirs('data')

# Download the raw data files
This may take some time

In [6]:
# download the files
url =  'https://zenodo.org/record/3228846/files/'

for subject in range(10):
    for session in range(3):
        ann = url + 'subject%02d_session%02d_ann.npy'  %(subject+1, session+1)
        dvs = url + 'subject%02d_session%02d_dvs.aedat'%(subject+1, session+1)
        emg = url + 'subject%02d_session%02d_emg.npy'  %(subject+1, session+1)

        wget.download(ann, 'rawData/')
        wget.download(dvs, 'rawData/')
        wget.download(emg, 'rawData/')

In [3]:
# general stuff
fs = 200  # sampling frequency of MYO
VERBOSE = True
data_dir = 'rawData/'  # change this if needed
classes = ['pinky', 'elle', 'yo', 'index', 'thumb']
classes_dict = {'pinky': 0, 'elle': 1, 'yo': 2, 'index': 3, 'thumb': 4}
classes_inv = {v: k for k, v in classes_dict.items()}

# Load all emg data into subject obects 
We keep the emg sessions of each subject separated so that we can load later the correct DVS data. 
The final dataset will have for each sample the metadata about subject id and session id (id needed to clean the data or simply do some subject-specific stuff)

Take a look at the Person object in [utils.py](./utils.py). Each subject has fields for eeg, dvs, spikes and metadata. The final dataset will have a more straightforward data structure. 

In [4]:
subjects = {}
names = [name for name in os.listdir(data_dir) if "emg" in name]
last_name = ''
all_emg = []
all_ann = []
for name in sorted(names):
    _emg = np.load(data_dir + '{}'.format(name)).astype('float32')
    _ann = np.concatenate([np.array(['none']), np.load(data_dir + '{}'.format(name.replace("emg","ann")))[:-1]])
    name = name.split('_emg')[0]
    subjects[name] = Person(name, _emg, _ann, classes=classes)
    if VERBOSE:
        print("Loaded {}: EMG = [{}] // ANN = [{}]".format(name, _emg.shape, len(_ann)))

print("Data Loaded! {} Subjects".format(len(subjects.keys())))

# separates data in correct trial type
for name, data in subjects.items():
    for _class in classes:
        _annotation = np.float32(data.ann == _class)
        derivative = np.diff(_annotation)/1.0
        begins = np.where(derivative == 1)[0]
        ends = np.where(derivative == -1)[0]
        for b, e in zip(begins, ends):
            _trials = data.emg[b:e]
            data.trials[_class].append(_trials / np.std(_trials))
            data.begs[_class].append(b)
            data.ends[_class].append(e)
print("Done sorting trials!")

Loaded subject01_session01: EMG = [(25290, 8)] // ANN = [25290]
Loaded subject01_session02: EMG = [(25089, 8)] // ANN = [25089]
Loaded subject01_session03: EMG = [(25244, 8)] // ANN = [25244]
Loaded subject02_session01: EMG = [(25077, 8)] // ANN = [25077]
Loaded subject02_session02: EMG = [(25331, 8)] // ANN = [25331]
Loaded subject02_session03: EMG = [(25318, 8)] // ANN = [25318]
Loaded subject03_session01: EMG = [(25315, 8)] // ANN = [25315]
Loaded subject03_session02: EMG = [(25306, 8)] // ANN = [25306]
Loaded subject03_session03: EMG = [(25328, 8)] // ANN = [25328]
Loaded subject04_session01: EMG = [(25315, 8)] // ANN = [25315]
Loaded subject04_session02: EMG = [(25328, 8)] // ANN = [25328]
Loaded subject04_session03: EMG = [(25242, 8)] // ANN = [25242]
Loaded subject05_session01: EMG = [(25358, 8)] // ANN = [25358]
Loaded subject05_session02: EMG = [(25330, 8)] // ANN = [25330]
Loaded subject05_session03: EMG = [(25332, 8)] // ANN = [25332]
Loaded subject06_session01: EMG = [(2499

In [5]:
# check that we get 5 trials per subject per gesture
for sub_name, sub_data in subjects.items():
    for _class, trials in sub_data.trials.items():
        assert (len(trials) == 5), "Something wrong with the number of trials!"
print("All good!")

All good!


Now every subject has different fields, each field is a dictionary and the keys are the gestures.
We have the following fields: begs (beginning time of trials), ends (end time of trials), trials (actual myo data) and others.

In [6]:
for subject, data in sorted(subjects.items()):

    # decoders takes some time since it is a very long recordings
    events = aedat2numpy(data_dir + subject +'_dvs.aedat')
    events = events[:, find_trigger(events[2]):]
    events[2] = events[2] / 1e3
    
    for gesture in classes:
        for trial in range(5):
            if VERBOSE:
                print("{} :: {} :: {}".format(subject, gesture, trial))
                
            # load beginning and end 
            b = np.array(data.begs[gesture][trial]) / fs
            e = np.array(data.ends[gesture][trial]) / fs

            # extract 2s frames
            frame_size = 2
            shift = 0.
            beginning = b + shift
            ending = beginning + frame_size

            # slice events
            sl = (events[2] > beginning) & (events[2] < ending)

            img = events[:, sl]

            data.x[gesture].append(img[0])
            data.y[gesture].append(img[1])
            data.ts[gesture].append(img[2] - img[2][0])  # reset timestamps to zero
            data.pol[gesture].append(img[3])


subject01_session01 :: pinky :: 0
subject01_session01 :: pinky :: 1
subject01_session01 :: pinky :: 2
subject01_session01 :: pinky :: 3
subject01_session01 :: pinky :: 4
subject01_session01 :: elle :: 0
subject01_session01 :: elle :: 1
subject01_session01 :: elle :: 2
subject01_session01 :: elle :: 3
subject01_session01 :: elle :: 4
subject01_session01 :: yo :: 0
subject01_session01 :: yo :: 1
subject01_session01 :: yo :: 2
subject01_session01 :: yo :: 3
subject01_session01 :: yo :: 4
subject01_session01 :: index :: 0
subject01_session01 :: index :: 1
subject01_session01 :: index :: 2
subject01_session01 :: index :: 3
subject01_session01 :: index :: 4
subject01_session01 :: thumb :: 0
subject01_session01 :: thumb :: 1
subject01_session01 :: thumb :: 2
subject01_session01 :: thumb :: 3
subject01_session01 :: thumb :: 4
subject01_session02 :: pinky :: 0
subject01_session02 :: pinky :: 1
subject01_session02 :: pinky :: 2
subject01_session02 :: pinky :: 3
subject01_session02 :: pinky :: 4


In [7]:
# checking that frames are 2 seconds (could be slightly less)
print(len(subjects['subject10_session03'].x['pinky'][2]))
print(np.max(subjects['subject10_session03'].ts['pinky'][1]))

32162
1.9999819999999993


# Spiking EMG

In [8]:
interpolation = 3500.
refractory = 0.000
th_up = th_dn = 0.05
preprocess = True
preprocess_window = 5
preprocess_exp = 0.3
n_ch = 8
fs = 200

In [9]:
for name, data in subjects.items(): 
    for gesture in classes:
        for trial in range(5):
            if VERBOSE:
                print(f"{name} || {gesture} || {trial}")
                
            _emg = data.trials[gesture][trial]
            _all_spikes, _all_ch, _all_pol = [], [], []
            
            for i, raw_ch in enumerate(_emg.T):  # each channel separately

                _t = np.arange(0, raw_ch.shape[0] / fs, 1. / fs) 

                spk_up, spk_dn = signal_to_spike_refractory(interpolation, _t, raw_ch, th_up, th_dn, refractory)

                _all_spikes.extend(spk_up)
                _all_spikes.extend(spk_dn)
                _all_ch.extend(np.ones((len(spk_up),)) * i )
                _all_ch.extend(np.ones((len(spk_dn),)) * i )
                _all_pol.extend(np.ones((len(spk_up),)))
                _all_pol.extend(np.zeros((len(spk_dn),)))
                
            _idx = np.argsort(_all_spikes)
            
            _to_add = np.array([np.array(_all_ch)[_idx], 
                                np.array(_all_spikes)[_idx], 
                                np.array(_all_pol)[_idx]])
            
            data.spk_trials[gesture].append(_to_add) 

subject01_session01 || pinky || 0
subject01_session01 || pinky || 1
subject01_session01 || pinky || 2
subject01_session01 || pinky || 3
subject01_session01 || pinky || 4
subject01_session01 || elle || 0
subject01_session01 || elle || 1
subject01_session01 || elle || 2
subject01_session01 || elle || 3
subject01_session01 || elle || 4
subject01_session01 || yo || 0
subject01_session01 || yo || 1
subject01_session01 || yo || 2
subject01_session01 || yo || 3
subject01_session01 || yo || 4
subject01_session01 || index || 0
subject01_session01 || index || 1
subject01_session01 || index || 2
subject01_session01 || index || 3
subject01_session01 || index || 4
subject01_session01 || thumb || 0
subject01_session01 || thumb || 1
subject01_session01 || thumb || 2
subject01_session01 || thumb || 3
subject01_session01 || thumb || 4
subject01_session02 || pinky || 0
subject01_session02 || pinky || 1
subject01_session02 || pinky || 2
subject01_session02 || pinky || 3
subject01_session02 || pinky || 4


Save this data structure with everything will make a simpler dataset in the following sections. This pkl file can also be found [here](https://www.dropbox.com/s/s861m5obkpc9y8u/10_people_dvs_emg_v3_spikes.pkl?dl=0)

In [10]:
pkl.dump(subjects, open('data/10_people_dvs_emg_v3_spikes.pkl', 'wb'))

# Full spiking dataset
The dataset contains spikes from both dvs and emg separated in N trials each of 2 seconds long, plus info about subject id and session id for each trial. The dataset is a dictionary with the following keys:
- **y**: array of size 1xN with the class (0->4).
- **sub**: array of size 1xN with the subject id (1->10).
- **sess**: array of size 1xN with the session id (1->3).
- **dvs**: list of length N, each object in the list is a 2d array of size 4xT_n where T_n is the number of events in the trial and the 4 dimensions rappresent: 0 -> addr_x, 1 -> addr_y, 2 -> timestamp, 3 -> polarity .
- **emg**: list of length N, each object in the list is a 2d array of size 3xT_n where T_n is the number of events in the trial and the 3 dimensions rappresent: 0 -> addr, 1 -> timestamp, 3 -> polarity.

In [11]:
all_sub_id = []
all_ses_id = []
all_y = []
all_dvs = []
all_emg = []

for name, data in subjects.items(): 
    _sub = int(name[7:9])
    _ses = int(name[17:19])
    for gesture in classes:
        for trial in range(5):
            all_y.append(classes_dict[gesture])
            all_sub_id.append(_sub)
            all_ses_id.append(_ses)
            all_emg.append(data.spk_trials[gesture][trial])
            _to_add = np.array([data.x[gesture][trial],
                                data.y[gesture][trial],
                                data.ts[gesture][trial],
                                data.pol[gesture][trial]])
            all_dvs.append(_to_add)

all_sub_id = np.array(all_sub_id)
all_ses_id = np.array(all_ses_id)
all_y = np.array(all_y)

In [12]:
print(len(all_dvs))
print(len(all_emg))
print(len(all_sub_id))

750
750
750


In [13]:
print(all_emg[10].shape)
print(all_dvs[12].shape)
print(all_y.shape)

(3, 29836)
(4, 40122)
(750,)


Saving the simple spiking dataset. This file can also be found [here](https://www.dropbox.com/s/u5ufys95bc4d81v/relax_spikes_2seconds.pkl?dl=0) 

In [14]:
pkl.dump({'y': all_y, 
          'sub': all_sub_id, 
          'ses': all_ses_id, 
          'dvs': all_dvs, 
          'emg': all_emg}, open('data/relax_spikes_2seconds.pkl', 'wb'))

In [18]:
import pickle as pkl
d = pkl.load(open('relax_spikes_2seconds.pkl', 'rb'))

In [21]:
a = d['emg']
print(list(set(a[0][2])))

[0.0, 1.0]
